---
title: "Example Notebooks of SPICE Toolkit" \
author: "Naoya Ozaki" \
date: "26 November 2022" \
output: "spice_example"

---

## 1. SPICE Setup
Import SPICE library and download generic kernels.
This process takes long time because of downloading the large file (~120MB).

In [8]:
using Downloads

current_directory = dirname(abspath(@__FILE__))
spice_directory = current_directory * "/../data/lib"

# Find naif0012.tls in the directory
is_downloaded = false
genker_naif0012 = ""
for (root, dirs, files) in walkdir(spice_directory)
    is_downloaded = any(occursin.("naif0012.tls", files))
    if is_downloaded
        genker_naif0012 = root * "/naif0012.tls"
        break
    end
end
if !is_downloaded
    mkdir(spice_directory*"/lsk")
    genker_naif0012 = Downloads.download("https://naif.jpl.nasa.gov/pub/naif/generic_kernels/lsk/naif0012.tls", spice_directory*"/lsk/naif0012.tls")
end

# Find gm_de431.tpc in the directory
is_downloaded = false
genker_gm_de431 = ""
for (root, dirs, files) in walkdir(spice_directory)
    is_downloaded = any(occursin.("gm_de431.tpc", files))
    if is_downloaded
        genker_gm_de431 = root * "/gm_de431.tpc"
        break
    end
end
if !is_downloaded
    mkdir(spice_directory * "/pck")
    genker_gm_de431 = Downloads.download("https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/gm_de431.tpc", spice_directory * "/pck/gm_de431.tpc")
end

# Find de440.bsp in the directory
is_downloaded = false
genker_de440 = ""
for (root, dirs, files) in walkdir(spice_directory)
    is_downloaded = any(occursin.("de440.bsp", files))
    if is_downloaded
        genker_de440 = root * "/de440.bsp"
        break
    end
end
if !is_downloaded
    mkdir(spice_directory * "/spk")
    mkdir(spice_directory * "/spk/planets")
    genker_de440 = Downloads.download("https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/de440.bsp", spice_directory * "/spk/planets/de440.bsp")
end


/Users/naoya/Git/NBodyPropagator.jl/work/../data/lib/lsk/naif0012.tls

In [2]:
import SPICE
# Load generic kernels
SPICE.furnsh(genker_naif0012) # Leap seconds kernel
SPICE.furnsh(genker_gm_de431) # Gravity Constant
SPICE.furnsh(genker_de440) # Planetary ephemeris kernel

UndefVarError: UndefVarError: genker_naif0012 not defined

In [3]:
# Parameter Setting
et0 = SPICE.str2et("2021/04/06 13:52:32 UTC") # Initial Epoch
etf = SPICE.str2et("2022/04/06 13:52:32 UTC") # Final Epoch
num_grid = 10000 # Number of Grid

# Pre-calculation
et_all = LinRange(et0,etf,num_grid)
x_moon = zeros(num_grid, 6)

# Iteration
for i = 1:num_grid
    x_moon[i,:], _ = SPICE.spkez(301, et_all[i], "ECLIPJ2000", "NONE", 399)
end

x_moon

SPICE.SpiceError: The variable that points to the leapseconds (DELTET/DELTA_AT) could not be located in the kernel pool.  It is likely that the leapseconds kernel has not been loaded.

In [4]:
using Plots
plot(x_moon[:, 1], x_moon[:, 2], x_moon[:, 3], xlim=[-5e5, 5e5], ylim=[-5e5, 5e5], zlim=[-5e5, 5e5], aspect_ratio=:equal)

UndefVarError: UndefVarError: x_moon not defined